## Import design libraries

In [1]:
import sys
sys.path.append('..')

In [2]:
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing

# Specify constant, everything in microns

In [3]:
tl_core = 20.
tl_gap = 12.
tl_ground = 10.

# Sample generation

## ground and grid, layer config

In [8]:
reload(gdspy)
reload(creator)
reload(elements)


resonator_core = 7
resonator_gap = 4
resonator_ground = 16

pad_offset = 800
pad_element_offset = 1000
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 450
num_couplers = 1


layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}
sample = creator.Sample('1Q_test',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=4.7e3
num_qubits_line = 3

central_line_y = sample.chip_geometry.sample_vertical_size/2

# ChichkovAB = elements.AirBridge(width=15, 
#                                 length=60, 
#                                 padsize=30, 
#                                 min_spacing = 30,
#                                 layer_configuration=sample.layer_configuration)

chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)




## microwave contact pads

In [9]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-'+str(pad_side_id), 
                       (pad_offset,  sample.chip_geometry.sample_vertical_size/2), np.pi, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size-pad_offset,  
                        sample.chip_geometry.sample_vertical_size/2), 0, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(3):
    pad = elements.Pad('pad-bottom-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1), pad_offset), 
                       -np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_bottom.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-top-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1),
                        sample.chip_geometry.sample_vertical_size-pad_offset), 
                        np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_top.append(pad)
    sample.add(pad)
    
p1 = pads_left[0]
p2 = pads_right[0]

# Coaxmons

In [10]:

##########################################################################3
# add coaxmons

shift=-1/10
shift1=shift+4/5
phi1=1/5


Couplers1q_corner=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=phi1,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=1/2, 
                                          coupler_type='coupler', w =7,g=10, s=4), #one upper
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=1,
                                          coupler_type='grounded',w=4,g=4)
]
Couplers1q_corner1=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2+1/8,
                                          coupler_type='grounded', w =4,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=phi1,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=1/2, 
                                          coupler_type='coupler', w =7,g=10, s=4), #one upper
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=-1/2-1/6,
                                          coupler_type=None,w=4,g=4)
]
# Couplers1q_corner1=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2+1/6,
#                                           coupler_type='grounded', w =4,g=4),
#           elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=phi1,
#                                           coupler_type=None, w =10,g=4),
#           elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=1/2, 
#                                           coupler_type='coupler', w =7,g=10, s=4), #one upper
#           elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1,
#                                           coupler_type=None, w =10,g=4),
#           elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0,
#                                           coupler_type=None, w =10,g=4),
#           elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=-1/2-1/6,
#                                           coupler_type=None,w=10,g=4)
# ]
Couplers2q_corner=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift1,arc_finish=-3/6+1/100+shift1,phi=1/7,
                                          coupler_type=None, w =10,g=10),
          elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift1,arc_finish=3/6-1/100+shift1,phi=phi1,
                                          coupler_type=None, w =10,g=10),
          elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift1,arc_finish=5/6-1/100+shift1,phi=1/2, 
                                          coupler_type='coupler', w =10,g=40), #one upper
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift1,arc_finish=1/6-1/100+1+shift1,phi=1-1/4-1/12,
                                          coupler_type='coupler', w =7,g=10, s=4), # resonator
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift1,arc_finish=1/6-1/100+shift1,phi=1/4,
                                          coupler_type=None, w =10,g=10),
          elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift1,arc_finish=-3/6-1/100+shift1,phi=0,
                                          coupler_type='grounded',w=5,g=10)
]
Couplers2q_line=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2,
                                          coupler_type=None, w =10,g=4),
          elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=phi1,
                                          coupler_type='coupler', w =7,g=10, s=4),# for resonator
          elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=1/2, 
                                          coupler_type=None, w =10,g=40), #one upper
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1,
                                          coupler_type='coupler', w =10,g=40),
          elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0,
                                          coupler_type='coupler', w =10,g=40),
          elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=1,
                                          coupler_type='grounded',w=4,g=4)
]
# Couplers2q_line_top=[elements.coaxmon.CoaxmonCoupler(arc_start=-1/6-1/100+shift,arc_finish=-3/6+1/100+shift,phi=-1/2,
#                                           coupler_type='grounded', w =4,g=4),
#           elements.coaxmon.CoaxmonCoupler(arc_start=1/6+1/100+shift,arc_finish=3/6-1/100+shift,phi=phi1,
#                                           coupler_type='coupler', w =7,g=10, s=4),# for resonator
#           elements.coaxmon.CoaxmonCoupler(arc_start=3/6+1/100+shift,arc_finish=5/6-1/100+shift,phi=1/2, 
#                                           coupler_type=None, w =10,g=40), #one upper
#           elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+1+shift,arc_finish=1/6-1/100+1+shift,phi=1,
#                                           coupler_type='coupler', w =10,g=40),
#           elements.coaxmon.CoaxmonCoupler(arc_start=-1/6+1/100+shift,arc_finish=1/6-1/100+shift,phi=0,
#                                           coupler_type='coupler', w =10,g=40),
#           elements.coaxmon.CoaxmonCoupler(arc_start=-5/6+1/100+shift,arc_finish=-3/6-1/100+shift,phi=1,
#                                           coupler_type=None,w=10,g=4)
# ]


jj_coaxmon = {'a1':30,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle_qubit':-np.pi/2-np.pi/3,
               'angle_JJ': 0,
               'length':10,
               'width':4}

# add coaxmons 1st line
for line_id in range(2):
    for qubit_id in range(3):
        jj_coaxmon_ = {k:v for k, v in jj_coaxmon.items()} 
        if qubit_id == 0:
            couplers_ = Couplers1q_corner

        elif qubit_id == 1:
            couplers_ = Couplers1q_corner1
            jj_coaxmon_['angle_qubit'] = -np.pi/2+np.pi/4
#             
        elif qubit_id == 2:
            couplers_ = Couplers2q_corner
            jj_coaxmon_['angle_qubit'] = np.pi/8
        else:
            couplers_ = Couplers2q_corner

            
        if qubit_id==2:
            x_shift = 200
        else:
            x_shift = 100
        if line_id == 0:
            transformations = {}
#             transformations = {'mirror':[(sample.chip_geometry.sample_horizontal_size/2+qubit_position_offset*(qubit_id-(num_qubits_line-1)/2),
#                         sample.chip_geometry.sample_vertical_size/2+2*qubit_position_offset*(line_id-0.5)),
#                        (sample.chip_geometry.sample_horizontal_size/2+qubit_position_offset*(qubit_id-(num_qubits_line-1)/2),
#                         sample.chip_geometry.sample_vertical_size/2+2*qubit_position_offset*(line_id-0.5)+10)]}
            coaxmon= elements.coaxmon.Coaxmon(name='Coaxmon',
                        center=(sample.chip_geometry.sample_horizontal_size/2+1.2*qubit_position_offset*(qubit_id-(num_qubits_line-1)/2)+x_shift,
                                sample.chip_geometry.sample_vertical_size/2+2*qubit_position_offset*(line_id-0.5)),
                        center_radius = 100,
                        inner_couplers_radius = 140,
                        outer_couplers_radius = 200,
                        inner_ground_radius = 230,
                        outer_ground_radius = 250,
                        layer_configuration = sample.layer_configuration,
                        Couplers=couplers_,jj_params= jj_coaxmon_,
                                  transformations=transformations,
                                  calculate_capacitance=True)
        else:
            transformations = {'rotate':[np.pi,(sample.chip_geometry.sample_horizontal_size/2+1.2*qubit_position_offset*(qubit_id-(num_qubits_line-1)/2)+x_shift,
                                sample.chip_geometry.sample_vertical_size/2+2*qubit_position_offset*(line_id-0.5))]}
            if qubit_id==2:
                coaxmon= elements.coaxmon.Coaxmon(name='Coaxmon',
                        center=(sample.chip_geometry.sample_horizontal_size/2+1.2*qubit_position_offset*(qubit_id-(num_qubits_line-1)/2)+x_shift,
                                sample.chip_geometry.sample_vertical_size/2+2*qubit_position_offset*(line_id-0.5)),
                        center_radius = 100,
                        inner_couplers_radius = 140,
                        outer_couplers_radius = 200,
                        inner_ground_radius = 230,
                        outer_ground_radius = 250,
                        layer_configuration = sample.layer_configuration,
                        Couplers=couplers_,jj_params= jj_coaxmon_,
                                  transformations=transformations,
                                  calculate_capacitance=True)


        sample.add(coaxmon)

sample.draw_design()

In [11]:
sample.draw_design()
sample.watch()

## couplers

In [11]:
jj_coupler = {'indent':40,
               'a1':0.243,
               'a2':0.243,
               'angle': 0}
fluxline_parameters_vertical={'w':4,
                's':4,
                'g':4,
                'length_x':20,
                'length_y':25,
                'width':4}
fluxline_parameters_horizontal={'w':4,
                's':4,
                'g':4,
                'length':30,
                'width':4}

# 1st horizontal line of couplers
for line_id in range(2):
    if line_id == 0:
        
        removing = {'left':9,'right':20,'up':0,'down':40}
    else:
        removing = {'left':10,'right':17,'up':0,'down':40}
        

################# vertical coupler
squid_coupler = {'a1':5,
               'b1':0.8,
               'a2':0.45,
               'b2':0.243,
               'c1':0.243,
               'c2':10,
               'angle':0,
               'side':'right',
               'point':((sample.qubits[5].get_terminals()['coupler2'].position[0]+
                         sample.qubits[2].get_terminals()['coupler2'].position[0])/2+25,
                        (sample.qubits[5].get_terminals()['coupler2'].position[1]+
                         sample.qubits[2].get_terminals()['coupler2'].position[1])/2  ),
                 'removing':{'left':0,'right':30,'up':10,'down':40}}
########### fix in the future, qubit4 should be qubit1 and qubit 9= qubit2
coupler_qq = elements.tqcoupler.MMCoupler('coupler',qubit1=sample.qubits[2],coupler1_name='coupler2',
                                       qubit2=sample.qubits[5],coupler2_name='coupler2',
                               core = 10,
                               gap = 25,
                               ground = 30,
                               layer_configuration = sample.layer_configuration,
                                jj_params=jj_coupler,squid_params=squid_coupler,fluxline=fluxline_parameters_vertical)
sample.add(coupler_qq)

In [14]:
sample.watch()

# Feedlines+Resonators

In [12]:
left_element = pads_left[0]
left_port = 'port'
ebanoe_govno = 0.5*(resonator_core+ resonator_ground+ tl_core + resonator_gap + resonator_gap + tl_gap + tl_gap +2*tl_ground)
for coupler_id in range(3):
    # coupler_start_x = sample.chip_geometry.sample_horizontal_size/2+qubit_position_offset*(coupler_id-(num_qubits_line-1)/2)-440
    if coupler_id == 2:
        start_shift=280
    else:
        start_shift=50
    coupler_start_x = list(sample.qubits[coupler_id].get_terminals()['coupler2'].position)[0]-start_shift
    # 2. Create main copler:
    total_length=0
    main_coupler = elements.CPWCoupler('TL-resonator coupler', [(coupler_start_x-coupler_length, central_line_y-ebanoe_govno+tl_ground + tl_gap +0.5*tl_core), 
                                                                (coupler_start_x, central_line_y-ebanoe_govno+tl_ground + tl_gap +0.5*tl_core)],
                                       [resonator_core, resonator_ground, tl_core],[resonator_gap, resonator_gap, tl_gap, tl_gap],
                                       tl_ground, sample.layer_configuration, r=100)
    sample.add(main_coupler)
    total_length = total_length + coupler_length

    # 3. Create fanout to create closed end of resonator
    fanout_for_closed_end = sample.fanout(o=main_coupler, port='port1', name='closed end resonator fanout', grouping=[1, 3])

    # 4. 
    g1 = sample.ground(o=fanout_for_closed_end, port='center', name='cl1', grounding_width=10, grounding_between = [(2,3)])

    # 6. Create closed meander of resonator
    if coupler_id == 2:
        length_right= 350
        length_left = 50
    else:
        length_right= 150
        length_left = 150
    closed_end_meander = sample.connect_meander(name='closed end', o1=fanout_for_closed_end, port1='up',
                                                meander_length=3900,
                                                length_left =length_left,
                                                length_right =length_right,
                                                first_step_orientation ='right',
                                                meander_orientation = 0,meander_type='flush')


    print(closed_end_meander.length)
    #                                                 end_point=(1500,1500),
    #                                                end_orientation=0)
    total_length = total_length + closed_end_meander.length
    # 7. Create grounding of resonator
    resonator_ground_ = sample.ground(o=closed_end_meander, port='port2', name='resonator ground', grounding_width=30, grounding_between = [(0,2)])

    # 8. Create fanout to create open end of resonator
    fanout_for_open_end = sample.fanout(o=main_coupler, port='port2', name='open end resonator fanout', grouping=[1, 3])

    #     # 9. 
    # g2 = sample.ground(o=fanout_for_open_end, port='center', name='cl2', grounding_width=10, grounding_between = [(0,1)])

        # 10. Create open end of resonator
    # open_end = elements.OpenEnd(name='open end',position=(fanout_for_open_end.get_terminals()['down'].position[0],
    #                                                                    fanout_for_open_end.get_terminals()['down'].position[1]-open_length),
    #                                          w=[resonator_core],
    #                                          s=[resonator_gap, resonator_gap],
    #                                          g=tl_ground,
    #                                          orientation=-np.pi/2,
    #                                          layer_configuration=sample.layer_configuration,
    #                                          h1=20,
    #                                           h2=10,
    #                                          )
    #     sample.add(open_end)
    #     # 11. Connect open end with the coupler part of the resonator
    #     open_end_resonator = sample.connect_cpw(fanout_for_open_end,open_end,'down','wide',name='right open end',points=[])
    #     total_length = total_length + open_end_resonator.length
    # res_params = total_length#,open_end_resonator.z0,open_end_resonator.cl)


    left_TL = sample.connect_cpw(o1=left_element, o2=g1, port1=left_port, port2='narrow', name='left TL'+str(coupler_id), 
                                 points=[])
    if coupler_id ==2:
        port1='coupler3'
    else:
        port1='coupler2'
    c1 = sample.connect_cpw(sample.qubits[coupler_id], fanout_for_open_end, port1=port1, port2='down', 
                       name='open end res'+str(coupler_id), 
                            points=sample.cpw_shift(sample.qubits[coupler_id],port1,-20))
    g2 = sample.ground(o=fanout_for_open_end, port='center', name='right grounding'+str(coupler_id), 
                           grounding_width=10, grounding_between = [(0,1), (3, 4)])

    left_element = g2
    left_port = 'narrow'

Meander
[(982.5, 2287.0), (982.5, 2237.0), (1132.5, 2237.0), (1132.5, 2167.0), (832.5, 2167.0), (832.5, 2097.0), (1132.5, 2097.0), (1132.5, 2027.0), (832.5, 2027.0), (832.5, 1957.0), (1132.5, 1957.0), (1132.5, 1887.0), (832.5, 1887.0), (832.5, 1817.0), (1132.5, 1817.0), (1132.5, 1747.0), (832.5, 1747.0), (832.5, 1677.0), (1132.5, 1677.0), (1132.5, 1607.0), (832.5, 1607.0), (832.5, 1537.0), (1132.5, 1537.0), (1132.5, 1537.0)]
3900.15000005
3900.15000005
Meander
[(1942.5, 2287.0), (1942.5, 2237.0), (2092.5, 2237.0), (2092.5, 2167.0), (1792.5, 2167.0), (1792.5, 2097.0), (2092.5, 2097.0), (2092.5, 2027.0), (1792.5, 2027.0), (1792.5, 1957.0), (2092.5, 1957.0), (2092.5, 1887.0), (1792.5, 1887.0), (1792.5, 1817.0), (2092.5, 1817.0), (2092.5, 1747.0), (1792.5, 1747.0), (1792.5, 1677.0), (2092.5, 1677.0), (2092.5, 1607.0), (1792.5, 1607.0), (1792.5, 1537.0), (2092.5, 1537.0), (2092.5, 1537.0)]
3900.15000005
3900.15000005
Meander
[(2772.5, 2287.0), (2772.5, 2237.0), (3122.5, 2237.0), (3122.5, 21

In [8]:
sample.draw_design()
sample.watch()

## connect contact pads to qubits

In [14]:
flux_pads = [
    (sample.qubits[0], pads_bottom[0]),
    (sample.qubits[1], pads_bottom[1]),
    (sample.qubits[2], pads_bottom[2]),
    (sample.qubits[3], pads_top[0]),
    (sample.qubits[4], pads_top[1]),
    (sample.qubits[5], pads_top[2]),
    
    ]

for coaxmon, pad in flux_pads:  
    flux_line_narrowing_position = np.asarray(coaxmon.terminals['flux_line'].position)
    direction = [np.cos(coaxmon.terminals['flux_line'].orientation), 
                 np.sin(coaxmon.terminals['flux_line'].orientation)]
    narrowing_length = 10.0
    flux_line_narrowing_position = flux_line_narrowing_position - np.asarray(direction)*narrowing_length/2
    flux_line_narrowing = elements.Narrowing(name='flux_line_narrowing', position=flux_line_narrowing_position, 
                                             orientation=coaxmon.terminals['flux_line'].orientation+np.pi, 
                                             w1=coaxmon.terminals['flux_line'].w, 
                                             s1=coaxmon.terminals['flux_line'].s, g1=coaxmon.terminals['flux_line'].g,
                                             w2=tl_core, s2=tl_gap, g2=tl_ground, 
                                             layer_configuration=sample.layer_configuration, length=narrowing_length)
    sample.add(flux_line_narrowing)

    sample.connect_cpw(pad, flux_line_narrowing, 'port', 'port2', 'flux_control5', points=[])

#sample.connect_cpw(pads_bottom[0], sample.qubits[6], 'port', 'flux_line', 'flux_control6', points=[])
#sample.connect_cpw(pads_bottom[2], sample.qubits[7], 'port', 'flux_line', 'flux_control7', points=[])
#sample.connect_cpw(pads_top[4], sample.qubits[8], 'port', 'flux_line', 'flux_control8', points=[])

ValueError: Too short segment in line to round corner with given radius

# Finalize

In [28]:
grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
                                  width=4, period=100)
sample.add(grid_ground)
sample.draw_design()

In [29]:
sample.draw_cap()

In [30]:
for i,qubit in enumerate(sample.qubits):
    sample.calculate_qubit_capacitance(cell= sample.qubit_cap_cells[i], qubit=sample.qubits[i],
                                       mesh_volume=10)

Gds file has been writen here:  C:\Users\Elena\git\QCreator\example_notebooks\1Q_test.gds
For 0 conductor total length is:  11984
For 1 conductor total length is:  4885
For 2 conductor total length is:  1579
For 3 conductor total length is:  1607
Sum : 20055
Data has been written into the file: C:\Users\Elena\git\QCreator\example_notebooks\mesh_4k_data


FileNotFoundError: [WinError 2] Не удается найти указанный файл

# Feedline

In [33]:
points = [p1.position, p2.position]
sample.generate_line('feedline', deepcopy(points), tl_core, tl_gap, tl_ground, None)
#('feedline', points, tl_core, tl_gap, tl_ground, sample.layer_configuration, R = 20)
sample.add(gdf.CPW('feedline', points, tl_core, tl_gap, tl_ground, None, sample.layer_configuration, R = 100))

AttributeError: 'Sample' object has no attribute 'generate_line'

In [32]:
feedline_bridge_params=creator.Bridges_over_line_param(distance=500, offset=30,
                    width=15, length=42, padsize=30, line_type=None)

sample.generate_array_of_bridges(deepcopy(points),feedline_bridge_params)

AttributeError: 'Sample' object has no attribute 'generate_array_of_bridges'

# Draw design

In [17]:
sample.draw_design()

C:\Users\Elena\Anaconda3\lib\site-packages\gdspy\path.py:828: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  "[GDSPY] Possible inconsistency found in "


In [17]:
sample.watch()

In [11]:
sample.write_to_gds()

Gds file has been writen here:  C:\Users\Elena\git\QCreator\example_notebooks\1Q_test.gds
